# Library page views interactive

This is another look at the small libraries pageview data, but this time combining multiple visualizations linked by interactive selections!

In [ ]:
import pandas as pd
import altair as alt
from altair import datum

## Read in library web site page views data


In [ ]:
pageviews = pd.read_csv('data/pageviews_2012_small.csv')
pageviews.head()

In [ ]:
len(pageviews)

In [ ]:
pageviews.dtypes

## Change ISO_time to a true date and time data type

In [ ]:
pageviews['timestamp'] = pd.to_datetime(pageviews.timestamp)
pageviews.dtypes

## MaxRowsError

Can take off limit for number of rows, but that's not a great idea because the notebook will have a Vega-Lite JSON specification (text) embedded for every output pane, which also includes the data, so you end up with huge notebooks!

Instead, we can specify that the outputs should just refer to a JSON file on your local drive and load in the data from there for each plot.

See the Altair tutorial notebook `03-Binning-and-aggregation` for more details.

In [ ]:
alt.data_transformers.enable('json')

## Filtering bar chart with time interval selection

In [ ]:
interval = alt.selection_interval(encodings=['x'],fields=['timestamp:T'])

timeline = alt.Chart(pageviews).mark_line().encode(
    x = alt.X('timestamp:T', timeUnit='yearmonthdate'),
    y = 'sum(visitors):Q',
    color = alt.value('darkgray')
).properties(
    height = 100
).add_selection(
    interval
)

bar = alt.Chart(pageviews).mark_bar().encode(
    x = 'sum(visitors)',
    y = 'country'
).transform_filter(
    interval
)

timeline & bar

## Bars filter timeline with single selection

In [ ]:
single = alt.selection_single(empty='all', fields=['lcc_description'])

bar = alt.Chart(pageviews).mark_bar().encode(
    x = 'sum(visitors)',
    y = alt.Y('lcc_description',
            sort=alt.EncodingSortField(
                field="visitors",
                op="sum",
                order="descending")
    ),
    color=alt.condition(single, alt.ColorValue("steelblue"), alt.ColorValue("lightgray")),
).properties(
    selection = single,
    height = 300
)

timeline = alt.Chart(pageviews).mark_line().encode(
    x = alt.X('timestamp:T', timeUnit='yearmonthdate'),
    y = 'sum(visitors):Q'
).transform_filter(
    single
).properties(
    height = 100
)

bar & timeline